# Notebook 01: Preprocesamiento de Imágenes CT Pulmonares

Este notebook cubre las técnicas de preprocesamiento para imágenes CT del dataset LUNA16:

1. **Carga de datos**: Lectura de archivos .mhd/.raw
2. **Segmentación pulmonar**: Extracción de la región pulmonar
3. **Normalización**: Conversión de valores HU a rango [0, 1]
4. **Experimentación**: Comparación de diferentes parámetros

---

## Configuración del entorno

Compatible con Google Colab y ejecución local.

In [2]:
# Detectar si estamos en Google Colab
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔵 Ejecutando en Google Colab")
    
    # Montar Google Drive (opcional, si tienes los datos allí)
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Instalar dependencias
    import subprocess
    paquetes = ['SimpleITK', 'scikit-image']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)
    
    # Clonar el repositorio con el código utils/
    # Descomenta y ajusta si tienes el código en un repo:
    # !git clone https://github.com/tu-usuario/tu-repo.git
    # %cd tu-repo
    
    # Si tienes los archivos utils/ en Drive:
    # sys.path.append('/content/drive/MyDrive/Imagenes Biomedicas')
    
else:
    print("🟢 Ejecutando localmente")
    # Añadir directorio padre al path para importar utils
    import os
    parent_dir = os.path.abspath('..')
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)

🟢 Ejecutando localmente


## Importar librerías

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure, morphology
from skimage.segmentation import clear_border

# Importar nuestros módulos personalizados
from utils import LUNA16DataLoader, LungPreprocessor, LungVisualizer

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Configuración de rutas

Define las rutas a tus datos. Ajusta según tu entorno:

In [ ]:
import os

if IN_COLAB:
    # ===== CONFIGURACIÓN PARA GOOGLE COLAB =====
    # Rutas en Google Drive (MODIFICA según tu estructura en Drive)
    DATA_PATH = '/content/drive/MyDrive/LUNA16/subset0'
    ANNOTATIONS_PATH = '/content/drive/MyDrive/LUNA16/annotations.csv'
    
else:
    # ===== CONFIGURACIÓN PARA EJECUCIÓN LOCAL =====
    
    # Buscar LUNA16 en el directorio padre de notebooks/
    # Estructura esperada:
    #   Imagenes Biomedicas/
    #   ├── notebooks/        <- estás aquí
    #   ├── LUNA16/
    #   │   ├── subset0/
    #   │   │   └── *.mhd, *.raw
    #   │   └── annotations.csv
    #   └── utils/
    
    project_root = os.path.abspath('..')  # Directorio padre de notebooks/
    
    possible_paths = [
        os.path.join(project_root, 'LUNA16', 'subset0'),  # Recomendado
        os.path.join(project_root, 'data', 'LUNA16', 'subset0'),
        os.path.join(project_root, 'data', 'subset0'),
        os.path.join(os.path.expanduser('~'), 'Desktop', 'LUNA16', 'subset0'),
        os.path.join(os.path.expanduser('~'), 'Documents', 'LUNA16', 'subset0'),
    ]
    
    DATA_PATH = None
    for path in possible_paths:
        if os.path.exists(path):
            DATA_PATH = path
            print(f"✅ Datos encontrados en: {DATA_PATH}")
            break
    
    # Si no se encontró, pedir al usuario
    if DATA_PATH is None:
        print("⚠️  No se encontraron datos LUNA16.")
        print("\n🔍 Ubicaciones buscadas:")
        for p in possible_paths:
            print(f"  - {p}")
        print("\n💡 Recomendación: Coloca la carpeta LUNA16 en el directorio raíz del proyecto:")
        print(f"   {os.path.join(project_root, 'LUNA16', 'subset0')}")
        print("\n📝 O ingresa la ruta completa a LUNA16/subset0:")
        DATA_PATH = input("Ruta: ").strip().strip('"').strip("'")
        
        if not os.path.exists(DATA_PATH):
            raise FileNotFoundError(f"❌ La ruta no existe: {DATA_PATH}")
    
    # Buscar archivo de anotaciones
    luna16_root = os.path.dirname(DATA_PATH)  # Directorio LUNA16
    possible_annotation_paths = [
        os.path.join(luna16_root, 'annotations.csv'),
        os.path.join(DATA_PATH, 'annotations.csv'),
        os.path.join(project_root, 'LUNA16', 'annotations.csv'),
    ]
    
    ANNOTATIONS_PATH = None
    for path in possible_annotation_paths:
        if os.path.exists(path):
            ANNOTATIONS_PATH = path
            print(f"✅ Anotaciones encontradas en: {ANNOTATIONS_PATH}")
            break
    
    if ANNOTATIONS_PATH is None:
        print("⚠️  No se encontró annotations.csv (opcional)")

# Detectar automáticamente el primer archivo .mhd disponible
if os.path.exists(DATA_PATH):
    mhd_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.mhd')]
    if mhd_files:
        EXAMPLE_SCAN = mhd_files[0]
        print(f"\n📁 Configuración final:")
        print(f"  - Directorio de datos: {DATA_PATH}")
        print(f"  - Anotaciones: {ANNOTATIONS_PATH if ANNOTATIONS_PATH else 'No disponible'}")
        print(f"  - Archivos .mhd encontrados: {len(mhd_files)}")
        print(f"  - Archivo de ejemplo a usar: {EXAMPLE_SCAN}")
    else:
        raise FileNotFoundError(f"❌ No se encontraron archivos .mhd en {DATA_PATH}")
else:
    raise FileNotFoundError(f"❌ El directorio no existe: {DATA_PATH}")

## 2. Cargar datos

Usamos la clase `LUNA16DataLoader` para cargar imágenes CT.

In [ ]:
# Inicializar el cargador
loader = LUNA16DataLoader(DATA_PATH, ANNOTATIONS_PATH)

# Cargar un escaneo de ejemplo
import os
scan_path = os.path.join(DATA_PATH, EXAMPLE_SCAN)

ct_scan, origin, spacing = loader.load_itk_image(scan_path)

print(f"\n📊 Información del volumen CT:")
print(f"  - Shape: {ct_scan.shape} (slices, height, width)")
print(f"  - Spacing: {spacing} mm (z, y, x)")
print(f"  - Origin: {origin} mm")
print(f"  - Rango HU: [{ct_scan.min():.1f}, {ct_scan.max():.1f}]")

## 3. Visualizar slice original

In [ ]:
# Seleccionar un slice del medio del volumen
slice_idx = ct_scan.shape[0] // 2
ct_slice = ct_scan[slice_idx]

plt.figure(figsize=(10, 10))
plt.imshow(ct_slice, cmap='bone')
plt.title(f'Slice {slice_idx} - Original (Unidades Hounsfield)')
plt.colorbar(label='HU')
plt.axis('off')
plt.show()

## 4. Segmentación pulmonar

### 4.1 Algoritmo básico

La segmentación pulmonar usa los siguientes pasos:

1. **Binarización**: Umbralización con threshold HU
2. **clear_border()**: Elimina aire externo
3. **Etiquetado**: Identifica componentes conectados
4. **Selección**: Mantiene las 2 regiones más grandes (pulmones izq/der)
5. **Morfología**: Suaviza con dilation → fill_holes → erosion

In [ ]:
# Crear instancia del preprocesador
preprocessor = LungPreprocessor()

# Segmentar pulmones con threshold por defecto (-320 HU)
lung_mask = preprocessor.segment_lung_mask(ct_slice, threshold=-320)

# Visualizar resultado
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(ct_slice, cmap='bone')
axes[0].set_title('CT Original')
axes[0].axis('off')

axes[1].imshow(lung_mask, cmap='gray')
axes[1].set_title('Máscara Pulmonar')
axes[1].axis('off')

axes[2].imshow(ct_slice, cmap='bone')
axes[2].imshow(lung_mask, cmap='Reds', alpha=0.3)
axes[2].set_title('Superposición')
axes[2].axis('off')

plt.tight_layout()
plt.show()

### 4.2 Experimentación: Comparación de thresholds

Diferentes valores de threshold usados en la literatura:

- **-604 HU**: s-mostafa-a/Luna16
- **-350 HU**: EliasVansteenkiste/dsb3
- **-320 HU**: Código actual (valor intermedio)

Vamos a comparar visualmente estos valores:

In [ ]:
# Configurar thresholds a probar
thresholds_to_test = {
    's-mostafa-a/Luna16 (-604 HU)': -604,
    'EliasVansteenkiste/dsb3 (-350 HU)': -350,
    'Código actual (-320 HU)': -320,
    'Personalizado (-400 HU)': -400,  # Puedes cambiar este valor
}

# Generar máscaras con cada threshold
masks = {}
for name, threshold in thresholds_to_test.items():
    masks[name] = preprocessor.segment_lung_mask(ct_slice, threshold=threshold)

# Visualizar comparación
fig, axes = plt.subplots(2, 2, figsize=(14, 14))
axes = axes.flatten()

for idx, (name, mask) in enumerate(masks.items()):
    axes[idx].imshow(ct_slice, cmap='bone')
    axes[idx].imshow(mask, cmap='Reds', alpha=0.3)
    axes[idx].set_title(name, fontsize=12)
    axes[idx].axis('off')
    
    # Calcular área segmentada
    area_pixels = np.sum(mask)
    area_mm2 = area_pixels * spacing[1] * spacing[2]
    axes[idx].text(10, 30, f'Área: {area_mm2:.0f} mm²', 
                   color='yellow', fontsize=10, weight='bold',
                   bbox=dict(boxstyle='round', facecolor='black', alpha=0.5))

plt.suptitle('Comparación de Thresholds para Segmentación Pulmonar', fontsize=16)
plt.tight_layout()
plt.show()

### 4.3 ¿Por qué usamos clear_border()?

**Problema**: Tanto el aire dentro de los pulmones como el aire fuera del cuerpo tienen valores HU similares (~-1000 HU).

**Solución**: `clear_border()` elimina todas las regiones que tocan los bordes de la imagen, lo cual típicamente corresponde al aire externo.

Visualicemos el efecto:

In [ ]:
# PASO 1: Binarización simple (SIN clear_border)
threshold = -320
binary_with_external_air = ct_slice < threshold

# PASO 2: Aplicar clear_border
binary_lungs_only = clear_border(binary_with_external_air.copy())

# PASO 3: Calcular diferencia (aire externo)
external_air_only = binary_with_external_air.astype(int) - binary_lungs_only.astype(int)

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(ct_slice, cmap='bone')
axes[0].imshow(binary_with_external_air, cmap='Reds', alpha=0.3)
axes[0].set_title('ANTES: Binarización simple\n(Incluye aire externo + pulmones)', fontsize=12)
axes[0].axis('off')

axes[1].imshow(ct_slice, cmap='bone')
axes[1].imshow(external_air_only, cmap='Greens', alpha=0.5)
axes[1].set_title('AIRE EXTERNO\n(Regiones eliminadas por clear_border)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(ct_slice, cmap='bone')
axes[2].imshow(binary_lungs_only, cmap='Blues', alpha=0.3)
axes[2].set_title('DESPUÉS: Solo pulmones\n(Aire externo eliminado)', fontsize=12)
axes[2].axis('off')

plt.suptitle('Efecto de clear_border() en Segmentación Pulmonar', fontsize=16)
plt.tight_layout()
plt.show()

print(f"\n📊 Estadísticas:")
print(f"  - Píxeles antes de clear_border: {np.sum(binary_with_external_air)}")
print(f"  - Píxeles después de clear_border: {np.sum(binary_lungs_only)}")
print(f"  - Píxeles eliminados (aire externo): {np.sum(external_air_only)}")

### 4.4 Algoritmo detallado de clear_border()

Veamos cómo funciona `clear_border()` paso a paso con un ejemplo sintético:

In [ ]:
# Crear imagen sintética 15×15 con 5 regiones
synthetic = np.zeros((15, 15), dtype=bool)

# Región 1: Toca borde superior (SERÁ ELIMINADA)
synthetic[0:3, 5:8] = True

# Región 2: Toca borde izquierdo (SERÁ ELIMINADA)
synthetic[5:8, 0:3] = True

# Región 3: Toca borde derecho (SERÁ ELIMINADA)
synthetic[10:13, 12:15] = True

# Región 4: NO toca bordes (SE MANTIENE - simula pulmón izquierdo)
synthetic[5:9, 5:8] = True

# Región 5: NO toca bordes (SE MANTIENE - simula pulmón derecho)
synthetic[5:9, 10:13] = True

# PASO 1: Crear máscara de bordes
border_mask = np.zeros_like(synthetic, dtype=bool)
border_mask[0, :] = True   # Borde superior
border_mask[-1, :] = True  # Borde inferior
border_mask[:, 0] = True   # Borde izquierdo
border_mask[:, -1] = True  # Borde derecho

# PASO 2: Etiquetar regiones conectadas
labeled, num_regions = measure.label(synthetic, return_num=True)

# PASO 3: Identificar qué regiones tocan los bordes
labels_at_border = np.unique(labeled[border_mask])
labels_at_border = labels_at_border[labels_at_border > 0]  # Excluir fondo (label=0)

# PASO 4: Eliminar esas regiones completas
result_manual = synthetic.copy()
for label in labels_at_border:
    result_manual[labeled == label] = False

# Comparar con resultado de clear_border
result_builtin = clear_border(synthetic)

# Visualizar
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Fila 1: Pasos del algoritmo
axes[0, 0].imshow(synthetic, cmap='gray')
axes[0, 0].set_title('ENTRADA\n5 regiones (3 tocan bordes)', fontsize=11)
axes[0, 0].axis('off')

axes[0, 1].imshow(labeled, cmap='nipy_spectral')
axes[0, 1].set_title(f'ETIQUETADO\n{num_regions} regiones identificadas', fontsize=11)
axes[0, 1].axis('off')

# Crear imagen que muestre bordes
border_highlight = synthetic.astype(float)
border_highlight[border_mask] = 0.5  # Resaltar bordes
axes[0, 2].imshow(border_highlight, cmap='RdYlGn')
axes[0, 2].set_title('DETECCIÓN DE BORDES\nRegiones que tocan bordes', fontsize=11)
axes[0, 2].axis('off')

# Fila 2: Resultado
axes[1, 0].imshow(result_manual, cmap='gray')
axes[1, 0].set_title('RESULTADO (Manual)\n2 regiones conservadas', fontsize=11)
axes[1, 0].axis('off')

axes[1, 1].imshow(result_builtin, cmap='gray')
axes[1, 1].set_title('RESULTADO (clear_border)\n2 regiones conservadas', fontsize=11)
axes[1, 1].axis('off')

# Diferencia (debería ser todo ceros)
diff = result_manual.astype(int) - result_builtin.astype(int)
axes[1, 2].imshow(diff, cmap='RdBu')
axes[1, 2].set_title(f'DIFERENCIA\nIguales: {np.all(diff == 0)}', fontsize=11)
axes[1, 2].axis('off')

plt.suptitle('Algoritmo de clear_border() - Ejemplo Sintético', fontsize=16)
plt.tight_layout()
plt.show()

print(f"\n🔍 Análisis del algoritmo:")
print(f"  - Regiones totales encontradas: {num_regions}")
print(f"  - Etiquetas que tocan bordes: {labels_at_border.tolist()}")
print(f"  - Regiones eliminadas: {len(labels_at_border)}")
print(f"  - Regiones conservadas: {num_regions - len(labels_at_border)}")
print(f"  - ✅ Implementación manual coincide con clear_border(): {np.all(result_manual == result_builtin)}")

## 5. Normalización de valores HU

Las imágenes CT están en Unidades Hounsfield (HU). Para entrenar redes neuronales, normalizamos a rango [0, 1].

**Ventana pulmonar típica**: [-1000, 400] HU
- -1000 HU: Aire
- -500 HU: Pulmón inflado
- 0 HU: Agua
- +400 HU: Hueso

In [ ]:
# Normalizar slice
ct_normalized = loader.normalize_hu(ct_slice, min_hu=-1000, max_hu=400)

# Aplicar máscara pulmonar
ct_lung_only = ct_normalized * lung_mask

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(ct_slice, cmap='bone')
axes[0].set_title(f'Original\nRango: [{ct_slice.min():.0f}, {ct_slice.max():.0f}] HU')
axes[0].axis('off')

axes[1].imshow(ct_normalized, cmap='bone')
axes[1].set_title(f'Normalizado\nRango: [{ct_normalized.min():.2f}, {ct_normalized.max():.2f}]')
axes[1].axis('off')

axes[2].imshow(ct_lung_only, cmap='bone')
axes[2].set_title('Solo región pulmonar\n(Normalizado + Máscara)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 6. CLAHE (Contrast Limited Adaptive Histogram Equalization)

CLAHE realza el contraste local, útil para visualizar nódulos de baja densidad (Ground Glass Opacities) en LDCT.

In [ ]:
# Aplicar CLAHE
ct_clahe = preprocessor.apply_clahe(ct_normalized, clip_limit=2.0, tile_size=(8, 8))

# Visualizar comparación
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(ct_normalized, cmap='bone')
axes[0].set_title('Normalizado (sin CLAHE)')
axes[0].axis('off')

axes[1].imshow(ct_clahe, cmap='bone')
axes[1].set_title('Con CLAHE (clip_limit=2.0)')
axes[1].axis('off')

# Aplicar máscara para comparar solo región pulmonar
ct_clahe_lung = ct_clahe * lung_mask
axes[2].imshow(ct_clahe_lung, cmap='bone')
axes[2].set_title('CLAHE + Máscara pulmonar')
axes[2].axis('off')

plt.suptitle('Efecto de CLAHE en Contraste Local', fontsize=16)
plt.tight_layout()
plt.show()

## 7. Pipeline completo de preprocesamiento

Función que combina todos los pasos:

In [ ]:
def preprocess_ct_slice(ct_slice, threshold=-320, apply_clahe_flag=False, 
                        min_hu=-1000, max_hu=400):
    """
    Pipeline completo de preprocesamiento para un slice CT
    
    Args:
        ct_slice: Slice 2D en HU
        threshold: Umbral para segmentación pulmonar
        apply_clahe_flag: Si aplicar CLAHE
        min_hu, max_hu: Ventana HU para normalización
        
    Returns:
        dict con resultados del preprocesamiento
    """
    # 1. Segmentar pulmones
    lung_mask = preprocessor.segment_lung_mask(ct_slice, threshold=threshold)
    
    # 2. Normalizar
    ct_normalized = loader.normalize_hu(ct_slice, min_hu=min_hu, max_hu=max_hu)
    
    # 3. CLAHE (opcional)
    if apply_clahe_flag:
        ct_processed = preprocessor.apply_clahe(ct_normalized)
    else:
        ct_processed = ct_normalized
    
    # 4. Aplicar máscara
    ct_lung_only = ct_processed * lung_mask
    
    return {
        'original': ct_slice,
        'normalized': ct_normalized,
        'processed': ct_processed,
        'lung_mask': lung_mask,
        'lung_only': ct_lung_only
    }

# Probar pipeline
results = preprocess_ct_slice(ct_slice, threshold=-320, apply_clahe_flag=True)

# Visualizar
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

images = [
    ('Original (HU)', results['original'], 'bone'),
    ('Normalizado [0,1]', results['normalized'], 'bone'),
    ('+ CLAHE', results['processed'], 'bone'),
    ('Máscara Pulmonar', results['lung_mask'], 'gray'),
    ('Región Pulmonar', results['lung_only'], 'bone'),
    ('Superposición', results['processed'], 'bone')
]

for idx, (title, img, cmap) in enumerate(images):
    axes[idx].imshow(img, cmap=cmap)
    if idx == 5:  # Última imagen: superposición
        axes[idx].imshow(results['lung_mask'], cmap='Reds', alpha=0.2)
    axes[idx].set_title(title, fontsize=12)
    axes[idx].axis('off')

plt.suptitle('Pipeline Completo de Preprocesamiento', fontsize=16)
plt.tight_layout()
plt.show()

print("✅ Pipeline de preprocesamiento completado")

## 8. Procesamiento de volumen completo

Aplicar preprocesamiento a todo el volumen 3D:

In [ ]:
def preprocess_volume(ct_volume, threshold=-320, apply_clahe_flag=False):
    """
    Preprocesa un volumen CT completo slice por slice
    
    Args:
        ct_volume: Array 3D (slices, height, width) en HU
        threshold: Umbral para segmentación
        apply_clahe_flag: Si aplicar CLAHE
        
    Returns:
        dict con volúmenes procesados
    """
    num_slices = ct_volume.shape[0]
    
    # Inicializar arrays de salida
    lung_masks = np.zeros_like(ct_volume, dtype=np.uint8)
    ct_processed = np.zeros_like(ct_volume, dtype=np.float32)
    
    print(f"Procesando {num_slices} slices...")
    
    for i in range(num_slices):
        if i % 20 == 0:
            print(f"  Slice {i}/{num_slices}")
        
        ct_slice = ct_volume[i]
        results = preprocess_ct_slice(ct_slice, threshold, apply_clahe_flag)
        
        lung_masks[i] = results['lung_mask']
        ct_processed[i] = results['lung_only']
    
    print("✅ Volumen procesado completamente")
    
    return {
        'lung_masks': lung_masks,
        'processed': ct_processed
    }

# Procesar volumen completo (puede tardar unos minutos)
# Descomenta para ejecutar:
# volume_results = preprocess_volume(ct_scan, threshold=-320, apply_clahe_flag=True)

## 9. Guardar resultados (opcional)

Guardar volúmenes preprocesados para uso posterior:

In [ ]:
# Ejemplo de cómo guardar resultados
# Descomenta para ejecutar:

# import SimpleITK as sitk

# # Convertir array a imagen SimpleITK
# lung_mask_volume = sitk.GetImageFromArray(volume_results['lung_masks'])
# lung_mask_volume.SetSpacing(spacing.tolist())
# lung_mask_volume.SetOrigin(origin.tolist())

# # Guardar
# output_path = 'lung_mask_processed.mhd'
# sitk.WriteImage(lung_mask_volume, output_path)
# print(f"✅ Máscara guardada en: {output_path}")

---

## Resumen

En este notebook has aprendido:

1. ✅ Cargar imágenes CT del dataset LUNA16
2. ✅ Segmentar pulmones usando umbralización y morfología
3. ✅ Entender y visualizar el efecto de `clear_border()`
4. ✅ Comparar diferentes valores de threshold
5. ✅ Normalizar valores HU a rango [0, 1]
6. ✅ Aplicar CLAHE para realce de contraste
7. ✅ Construir un pipeline completo de preprocesamiento

**Próximos pasos**: Ver notebook `02_visualizacion.ipynb` para explorar técnicas de visualización avanzadas.